使用 os.environ 建置 HF_TOKEN 作為環境變數

In [2]:
import os
HF_TOKEN = "--- personal HF_TOKEN ---"
os.environ["HF_TOKEN"] = HF_TOKEN

讀入要訓練的數據集
src: https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0

In [3]:
from datasets import load_dataset, DatasetDict
common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "ja", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "ja", split="test", use_auth_token=True)
common_voice = common_voice.remove_columns(
    ["accent",
     "age",
     "client_id",
     "down_votes",
     "gender",
     "locale",
     "path",
     "segment",
     "up_votes"]
     ) 

print(common_voice)

c:\Users\mycena\.conda\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\mycena\.conda\envs\llm\Lib\site-packages\datasets\load.py:2516: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
c:\Users\mycena\.conda\envs\llm\Lib\site-packages\datasets\load.py:1461: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load 

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 10990
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 4604
    })
})


從 transformers 套件讀入 whisper 套件：FeatureExtractor、Tokenizer、Processor

In [4]:
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
from transformers import WhisperProcessor

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Japanese", task="transcribe")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small", language="Japanese", task="transcribe") 
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Japanese", task="transcribe") 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


語音數據前處理：
(1) 採樣率（sampling）
(2) 使用 feature_extractor 來產生 Mel spectrum 
(3) 將輸入label編碼

In [5]:
def prepare_dataset(batch):
    """
    Prepare audio data to be suitable for Whisper AI model.
    """
    # (1) load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # (2) compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # (3) encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [6]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))
common_voice = common_voice.map(
    prepare_dataset,
    remove_columns=common_voice.column_names["train"] # -> https://discuss.huggingface.co/t/nameerror-name-feature-extractor-is-not-defined/78537/3
    # remove_columns=common_voice.column_names["train"], num_proc=2
)


Map: 100%|██████████| 4604/4604 [01:45<00:00, 43.73 examples/s]


關於 Dataset 的 padding

In [7]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch
    

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

關於訓練時的評估

In [10]:
import evaluate

metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == -100] = tokenizer.pad_token_id #若是 -100 則換成 pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)   # 在計算 metrics 時忽略一些特別的 token
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True) # 在計算 metrics 時忽略一些特別的 token 

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


設定 pretrained 模型

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

model.generation_config.language = 'ja'
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [11]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-japanese",  # 可自己定義，為結果產生檔案的資料夾
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # batch 大小每減少 2 倍，增加 2 倍
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [12]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

processor.save_pretrained(training_args.output_dir)

trainer.train()

c:\Users\mycena\.conda\envs\llm\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  0%|          | 0/4000 [00:00<?, ?it/s]c:\Users\mycena\.conda\envs\llm\Lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.


{'loss': 2.6847, 'grad_norm': 96.16582489013672, 'learning_rate': 4.2000000000000006e-07, 'epoch': 0.04}


  1%|▏         | 50/4000 [03:32<4:48:28,  4.38s/it]

{'loss': 2.0588, 'grad_norm': 20.804677963256836, 'learning_rate': 9.200000000000001e-07, 'epoch': 0.07}


  2%|▏         | 75/4000 [05:25<4:59:42,  4.58s/it]

...

                                                     
100%|██████████| 4000/4000 [8:43:19<00:00,  4.43s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}


{'eval_loss': 0.3166385293006897, 'eval_wer': 73.31454584868848, 'eval_runtime': 2157.3112, 'eval_samples_per_second': 2.134, 'eval_steps_per_second': 0.267, 'epoch': 5.82}


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].
100%|██████████| 4000/4000 [8:43:41<00:00,  7.86s/it]


{'train_runtime': 31421.0943, 'train_samples_per_second': 2.037, 'train_steps_per_second': 0.127, 'train_loss': 0.15366882096230983, 'epoch': 5.82}


TrainOutput(global_step=4000, training_loss=0.15366882096230983, metrics={'train_runtime': 31421.0943, 'train_samples_per_second': 2.037, 'train_steps_per_second': 0.127, 'train_loss': 0.15366882096230983, 'epoch': 5.82})

上傳訓練完成的模型

In [16]:
from huggingface_hub import create_repo , HfApi 

api = HfApi(token=HF_TOKEN)

api.create_repo(
    repo_id = f"{自己的帳戶名稱}/{自己取的模型名稱}",
    repo_type="model",
    exist_ok=True,
)

api.upload_folder(
    folder_path="{放檔案的資料夾}",
    repo_id=f"soaring0616/whisper-small-japanese",
)

CommitInfo(commit_url='https://huggingface.co/soaring0616/whisper-small-japanese/commit/b5f5d2563d54951a7bddd20ca457ca55c66a42b8', commit_message='Upload folder using huggingface_hub', commit_description='', oid='b5f5d2563d54951a7bddd20ca457ca55c66a42b8', pr_url=None, pr_revision=None, pr_num=None)

取下剛剛的模型來玩！

In [21]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor

model = WhisperForConditionalGeneration.from_pretrained("{自己的帳戶名稱}/{自己取的模型名稱}")
processor = WhisperProcessor.from_pretrained("{自己的帳戶名稱}/{自己取的模型名稱}")

# model = WhisperForConditionalGeneration.from_pretrained("soaring0616/whisper-small-japanese")
# processor = WhisperProcessor.from_pretrained("soaring0616/whisper-small-japanese")

c:\Users\mycena\.conda\envs\llm\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mycena\.cache\huggingface\hub\models--soaring0616--whisper-small-japanese. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fi

In [3]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="{自己的帳戶名稱}/{自己取的模型名稱}")
# pipe = pipeline(model="soaring0616/whisper-small-japanese") 

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe, 
    inputs=gr.Audio(sources=["microphone"], type="filepath"), 
    outputs="text",
    title="Whisper Small Japanese",
    description="Realtime demo for Japanese speech recognition using a fine-tuned Whisper small model.",
)

iface.launch()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
